# VacationPy


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_main_df = pd.read_csv("../Python_API/output_data/cities.csv")
weather_main_df

,City,Country,Lat,Lng,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date(dt)
0,pilar,BR,-9.60,-35.96,75.20,88,75,1.12,1591750991
1,kapaa,US,22.08,-159.32,84.20,70,40,16.11,1591750991
2,mount gambier,AU,-37.83,140.77,44.60,87,0,8.05,1591750991
3,san cristobal,VE,7.77,-72.22,86.00,48,75,21.92,1591750992
4,kaitangata,NZ,-46.28,169.85,55.51,58,15,12.06,1591750992
...,...,...,...,...,...,...,...,...,...
559,tarko-sale,RU,64.92,77.78,63.00,78,100,7.18,1591751105
560,panama city,PA,8.99,-79.52,78.80,88,75,3.36,1591750935
561,san ramon de la nueva oran,AR,-23.13,-64.33,63.12,86,100,1.59,1591751105
562,ames,US,42.03,-93.62,75.20,78,75,9.17,1591750912


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps.
gmaps.configure(api_key= g_key)

# Use the Lat and Lng as locations and Humidity as the weight. ph/Variables for holding Latitude, Longitude and Humidity (%) 

locations = weather_main_df[["Lat","Lng"]]
humidity = weather_main_df["Humidity (%)"]


# Add Heatmap layer to map.
fig = gmaps.figure(center=(45.0, -5.0), zoom_level=2)

heat_layer= gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down the cities to fit weather conditions. 
#A max temperature lower than 80 degrees but higher than 70. Wind speed less than 10 mph. Zero cloudiness.
#Drop any rows that don't contain all three conditions. 
#Drop any rows will null values.

ideal_city_df = weather_main_df.loc[(weather_main_df["Wind Speed (mph)"] < 10) & (weather_main_df["Max Temperature (F)"] >= 70) \
                & (weather_main_df["Max Temperature (F)"] <= 80) & (weather_main_df["Cloudiness (%)"] == 0)].dropna()

ideal_city_df




,City,Country,Lat,Lng,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date(dt)
11,piranhas,BR,-16.43,-51.82,72.90,58,0,2.80,1591750993
17,albany,US,42.60,-73.97,73.99,50,0,4.00,1591750994
44,sao joao da barra,BR,-21.64,-41.05,72.05,89,0,9.37,1591750998
56,maragogi,BR,-9.01,-35.22,75.74,87,0,4.85,1591751017
69,vila velha,BR,-20.33,-40.29,71.60,88,0,3.36,1591750891
71,port alfred,ZA,-33.59,26.89,73.00,36,0,1.99,1591751019
112,carnarvon,AU,-24.87,113.63,70.92,32,0,3.00,1591751027
166,asyut,EG,27.18,31.18,71.60,53,0,6.93,1591751048
172,corinto,GR,37.94,22.96,70.00,62,0,8.48,1591750946
185,canyon,US,34.98,-101.92,77.00,29,0,4.00,1591751022


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store into variable named hotel_df.

hotel_df = ideal_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df 

,City,Country,Lat,Lng,Hotel Name
11,piranhas,BR,-16.43,-51.82,
17,albany,US,42.60,-73.97,
44,sao joao da barra,BR,-21.64,-41.05,
56,maragogi,BR,-9.01,-35.22,
69,vila velha,BR,-20.33,-40.29,
71,port alfred,ZA,-33.59,26.89,
112,carnarvon,AU,-24.87,113.63,
166,asyut,EG,27.18,31.18,
172,corinto,GR,37.94,22.96,
185,canyon,US,34.98,-101.92,


In [6]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#geocoordinates
params = {"type": "hotel",
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key}


for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}")
    response = requests.get(url, params=params).json()
    results = response["results"]
      
      
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}")
          
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
            
    except (KeyError, IndexError):
        print("Missing Fields/Results...skipping")
    print("-----------------------------------------")
            
    #time.sleep(1)

print("________________The End__________________")

#hotel_df 

Retrieving Results for Index 11: piranhas
Closest hotel in piranhas is Tropical Palace Hotel
-----------------------------------------
Retrieving Results for Index 17: albany
Missing Fields/Results...skipping
-----------------------------------------
Retrieving Results for Index 44: sao joao da barra
Closest hotel in sao joao da barra is Pousada Portobello
-----------------------------------------
Retrieving Results for Index 56: maragogi
Closest hotel in maragogi is Salinas do Maragogi All Inclusive Resort
-----------------------------------------
Retrieving Results for Index 69: vila velha
Closest hotel in vila velha is Golden Tulip Porto Vitória
-----------------------------------------
Retrieving Results for Index 71: port alfred
Closest hotel in port alfred is The Halyards Hotel
-----------------------------------------
Retrieving Results for Index 112: carnarvon
Closest hotel in carnarvon is The Gascoyne Hotel
-----------------------------------------
Retrieving Results for Index

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))